In [33]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [131]:
class Value:
    def __init__(self, data, __children={}, _op="", label=""):
        self.data = data
        self.grad = 0
        self._prev = set(__children)
        self._op = _op
        self._backward = lambda: None
        self.label = label
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    def __neg__(self):
        return self * -1
    def __sub__(self, other):
        return self + (-other)
    def __rmul__(self, other):
        return self * other
    def __radd__(self, other): # other + self
        return self + other
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out = Value(self.data * other.data, (self, other), "*")
        out._backward = _backward
        return out
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/floa powers for now"
        out = Value(self.data**other, (self, ), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out
    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return self * other**-1
    def tanh(self):
        def _backward():
            self.grad += (1 - t**2) * out.grad
        n = self.data
        t = (math.exp(2*n) - 1)/(math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')
        out._backward = _backward
        return out
    def exp(self):
        def _backward():
            self.grad += out.data * out.grad
        n = self.data
        out = Value(math.exp(n), (self, ), 'exp')
        out._backward = _backward
        return out
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if(v not in visited):
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0

        for node in reversed(topo):
            node._backward()

In [132]:
from graphviz import Digraph

def trace(root):
    # build a set of all the nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if(v not in nodes):
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(v=root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if(n._op):
            dot.node(name = uid + n._op, label = n._op)
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
        
    return dot

In [133]:
import torch
import random

x1 = torch.Tensor([2.0]).double(); x1.requires_grad=True
w1 = torch.Tensor([0.0]).double(); w1.requires_grad=True
x2 = torch.Tensor([-3.0]).double(); x2.requires_grad=True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad=True
b = torch.Tensor([6.8813735870195432]).double(); b.requires_grad=True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()



0.9991497879990848


In [148]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1)) # bias controlling overall trigger happiness
    def __call__(self, x):
        wx_zip = zip(self.w, x) #takes 2 iternates and creates a new interator which iterates over tuples of corresponding entries
        act = sum((wi*xi for wi, xi in zip(self.w, x)),self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b] #list of neurons + list of bias params

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
                       
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
                       

In [169]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=-0.6632994245955616)

In [170]:
len(n.parameters())

41

In [335]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]


In [336]:
for k in range(1000):
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    for p in n.parameters():
        p.data += -0.05 * p.grad 
    print(k, loss.data)

0 8.387546500854191e-23
1 8.387546500854191e-23
2 8.387546500854191e-23
3 8.387546500854191e-23
4 8.387546500854191e-23
5 8.387546500854191e-23
6 8.387546500854191e-23
7 8.387546500854191e-23
8 8.387546500854191e-23
9 8.387546500854191e-23
10 8.387546500854191e-23
11 8.387546500854191e-23
12 8.387546500854191e-23
13 8.387546500854191e-23
14 8.387546500854191e-23
15 8.387546500854191e-23
16 8.387546500854191e-23
17 8.387546500854191e-23
18 8.387546500854191e-23
19 8.387546500854191e-23
20 8.387546500854191e-23
21 8.387546500854191e-23
22 8.387546500854191e-23
23 8.387546500854191e-23
24 8.387546500854191e-23
25 8.387546500854191e-23
26 8.387546500854191e-23
27 8.387546500854191e-23
28 8.387546500854191e-23
29 8.387546500854191e-23
30 8.387546500854191e-23
31 8.387546500854191e-23
32 8.387546500854191e-23
33 8.387546500854191e-23
34 8.387546500854191e-23
35 8.387546500854191e-23
36 8.387546500854191e-23
37 8.387546500854191e-23
38 8.387546500854191e-23
39 8.387546500854191e-23
40 8.38754

478 8.387546500854191e-23
479 8.387546500854191e-23
480 8.387546500854191e-23
481 8.387546500854191e-23
482 8.387546500854191e-23
483 8.387546500854191e-23
484 8.387546500854191e-23
485 8.387546500854191e-23
486 8.387546500854191e-23
487 8.387546500854191e-23
488 8.387546500854191e-23
489 8.387546500854191e-23
490 8.387546500854191e-23
491 8.387546500854191e-23
492 8.387546500854191e-23
493 8.387546500854191e-23
494 8.387546500854191e-23
495 8.387546500854191e-23
496 8.387546500854191e-23
497 8.387546500854191e-23
498 8.387546500854191e-23
499 8.387546500854191e-23
500 8.387546500854191e-23
501 8.387546500854191e-23
502 8.387546500854191e-23
503 8.387546500854191e-23
504 8.387546500854191e-23
505 8.387546500854191e-23
506 8.387546500854191e-23
507 8.387546500854191e-23
508 8.387546500854191e-23
509 8.387546500854191e-23
510 8.387546500854191e-23
511 8.387546500854191e-23
512 8.387546500854191e-23
513 8.387546500854191e-23
514 8.387546500854191e-23
515 8.387546500854191e-23
516 8.387546

958 8.387546500854191e-23
959 8.387546500854191e-23
960 8.387546500854191e-23
961 8.387546500854191e-23
962 8.387546500854191e-23
963 8.387546500854191e-23
964 8.387546500854191e-23
965 8.387546500854191e-23
966 8.387546500854191e-23
967 8.387546500854191e-23
968 8.387546500854191e-23
969 8.387546500854191e-23
970 8.387546500854191e-23
971 8.387546500854191e-23
972 8.387546500854191e-23
973 8.387546500854191e-23
974 8.387546500854191e-23
975 8.387546500854191e-23
976 8.387546500854191e-23
977 8.387546500854191e-23
978 8.387546500854191e-23
979 8.387546500854191e-23
980 8.387546500854191e-23
981 8.387546500854191e-23
982 8.387546500854191e-23
983 8.387546500854191e-23
984 8.387546500854191e-23
985 8.387546500854191e-23
986 8.387546500854191e-23
987 8.387546500854191e-23
988 8.387546500854191e-23
989 8.387546500854191e-23
990 8.387546500854191e-23
991 8.387546500854191e-23
992 8.387546500854191e-23
993 8.387546500854191e-23
994 8.387546500854191e-23
995 8.387546500854191e-23
996 8.387546

In [337]:
ypred

[Value(data=0.9999999999994542),
 Value(data=-0.9999999999935048),
 Value(data=-0.9999999999935674),
 Value(data=0.9999999999998972)]